In [ ]:
from gurobipy import *
from itertools import product

In [ ]:
def printSolution(m):
    line = " ||-----|-----|-----||-----|-----|-----||-----|-----|-----||"
    boldline = " ||=======================================================||"
    print "\n",boldline
    position = 0
    row = 0
    #print '\nOptimal flows:'
    for v in m.getVars():
        if v.X > 0:
            if position % 3 == 0:
                print " || ",
            else:
                print " | ",
            print int(v.VarName[3]),
            position +=1
            if position == 9:
                position = 0
                print " || "
                row += 1
                if row %3 == 0:
                    print boldline
                else:
                    print line

In [ ]:
def printPuzzle(s):
    line = " ||-----|-----|-----||-----|-----|-----||-----|-----|-----||"
    boldline = " ||=======================================================||"
    print "\n",boldline
    position = 0
    row = 0
    #print '\nOptimal flows:'
    for num in s:
        if num == "-":
            num = " "
        if position % 3 == 0:
            print " || ",
        else:
            print " | ",
        print num,
        position +=1
        if position == 9:
            position = 0
            print " || "
            row += 1
            if row %3 == 0:
                print boldline
            else:
                print line

## The make_boxes creates the representation of the 9 "big" boxes in the 
## puzzle, numbered like a tic-tac-toe grid.  Each "big" boxes has 9 cells. 
!['big_boxes'](big_boxes.png)
### For example, box 4 has these 9 (i,j) cells (where i = row and j = column):
### [(4, 1), (4, 2), (4, 3), (5, 1), (5, 2), (5, 3), (6, 1), (6, 2), (6, 3)]

In [ ]:
def make_boxes():
    box = {}
    boxn = 1
    lev = [[1,2,3],[4,5,6],[7,8,9]]
    for i in range(3):
        for j in range(3):
            box[boxn] = list(product(lev[i],lev[j]))
            boxn += 1
    return box

In [ ]:
# The puzzle examples

def get_puzzle(p):

    if p == 1:
        return ("-8--9--6-"
                "-7-----5-"
                "--35-71--"
                "85--1--29"
                "---------"
                "13--2--76"
                "--81-47--"
                "-1-----3-"
                "-2--8--1-")
    elif p == 2:
        return ("72--39--6"
                "----5-3-1"
                "-----7--5"
                "------1--"
                "9-6-----2"
                "-3----5--"
                "37---48--"
                "6-897--3-"
                "5--------")
    elif p == 3:
        return ("8-9-73562"
                "37-5694--"
                "654182-9-"
                "----5---9"
                "7-12-8--4"
                "-35--1-7-"
                "187-4--25"
                "54-927816"
                "-2681-7-3")
    elif p == 4:
        return ("2935-4---"
                "---3-7-95"
                "5------8-"
                "--58---3-"
                "1---4-8--"
                "42895361-"
                "74----5-1"
                "---1--9--"
                "85-429---")
    elif p == 5:
        return ("-2-6795--"
                "--7-18-9-"
                "----2-17-"
                "--8-4--6-"
                "----5----"
                "-6--9-8--"
                "-74-6----"
                "-1-53-9--"
                "--2784-1-")


    elif p == 98:
        return ("123456789"
                "---------"
                "---------"
                "---------"
                "---------"
                "---------"
                "---------"
                "---------"
                "---------")
    elif p == 99:
        return ("1--------"
                "-2-------"
                "--3------"
                "---4-----"
                "----5----"
                "-----6---"
                "------7--"
                "-------8-"
                "--------9")


    elif p == "bad":
        return ("77-3-1--8"
                "--64-85--"
                "-4--2--6-"
                "---------"
                "4-------1"
                "517---284"
                "2--7-6--3"
                "631---475"
                "----4----")
    else:
        return ("7--3-1--8"
                "--64-85--"
                "-4--2--6-"
                "---------"
                "4-------1"
                "517---284"
                "2--7-6--3"
                "631---475"
                "----4----")

In [ ]:
def solve(s, detail):
    n = 9
    vals = range(1,n+1) # [1,2,3,4,5,6,7,8,9]
    boxes = make_boxes()
    
    # variable description: There are 9^3 (729) variables.
    # x[row,col,digit] = 1  if part of the solution
    #                  = 0  otherwise
    #
    # example:
    # x[2,4,7] = 1  Means row 2, col 3 is a 7.
    # x[2,4,7] = 0  Means row 2, col 3 is NOT a 7.
    
    # Create optimization model
    m = Model('Sudoku')
    m.setParam('OutputFlag', detail)
    
    # define the variables 
    x = {}
    for i in vals:
        for j in vals:
            for k in vals:
                x[i,j,k] = m.addVar(obj=0,vtype=GRB.BINARY,name='x%d%d%d' % (i,j,k))
    m.update()

    #Set up constraints
    
    # in each row, each number is used only once
    for i in vals:
        for k in vals:
            m.addConstr(quicksum([x[i,j,k] for j in vals]) == 1, 'row%d%d' % (i,k))

    # in each column, each number is used only once
    for j in vals:
        for k in vals:
            m.addConstr(quicksum([x[i,j,k] for i in vals]) == 1, 'col%d%d' % (j,k))

    # in each "big" box, each number is used only once
    for b in boxes:
        for k in vals:
            m.addConstr(quicksum([x[i,j,k] for (i,j) in boxes[b]]) == 1, 'box%d%d' % (b,k))
    
    # each "little" box must have a number in it
    for i in vals:
        for j in vals:
            m.addConstr(quicksum([x[i,j,k] for k in vals]) == 1, 'cell%d%d%d' % (i,j,k))

    # Hard set the values that are known
    c = 0
    for i in vals:
        for j in vals:
            if s[c] != '-':
                p = int(s[c])
                for k in vals:
                    if k == p:
                        m.addConstr(x[i,j,k] == 1, 'init%d%d%d' % (i,j,k))
            c += 1

    # Compute optimal solution
    m.optimize()

    m.write('sudoku.lp')    

    if m.status == GRB.status.OPTIMAL:
        printSolution(m)
    else:
        print 'Bummer no solution, status code:',m.status
    
    if detail:    
        print('m.getVars()',m.getVars())
    
    return m

In [ ]:
puzzle = get_puzzle(1)
printPuzzle(puzzle)

In [ ]:
detail = False
m = solve(puzzle,detail)